# 🔐 JWT (JSON Web Token) 认证详解

> **JWT** 是现代 Web 应用中最流行的无状态认证方案。
> 
> 本 Notebook 将深入讲解 JWT 的原理、结构、使用方法以及安全注意事项。

---

## 📚 目录

1. [什么是 JWT？](#什么是-jwt)
2. [JWT 的结构](#jwt-的结构)
3. [JWT vs Session 认证](#jwt-vs-session-认证)
4. [PyJWT 库使用](#pyjwt-库使用)
5. [JWT 最佳实践](#jwt-最佳实践)
6. [常见安全问题](#常见安全问题)

## 🤔 什么是 JWT？

**JWT (JSON Web Token)** 是一种开放标准 (RFC 7519)，用于在各方之间安全地传输信息。

### 核心特点

| 特点 | 说明 |
|------|------|
| **无状态** | 服务器不需要存储会话信息 |
| **自包含** | 令牌本身包含所有必要信息 |
| **可验证** | 使用签名确保数据完整性 |
| **跨域友好** | 可在不同域之间传递 |

### 典型使用场景

```
┌──────────┐     1. 登录请求      ┌──────────┐
│          │ ──────────────────> │          │
│  客户端   │                     │  服务器   │
│          │ <────────────────── │          │
└──────────┘     2. 返回 JWT      └──────────┘
      │
      │ 3. 后续请求携带 JWT
      │    Authorization: Bearer <token>
      ▼
┌──────────┐
│  服务器   │ 4. 验证 JWT，返回数据
└──────────┘
```

## 🧬 JWT 的结构

JWT 由三部分组成，用 `.` 分隔：

```
xxxxx.yyyyy.zzzzz
  │      │      │
  │      │      └── Signature (签名)
  │      └── Payload (载荷)
  └── Header (头部)
```

### 1️⃣ Header（头部）
```json
{
  "alg": "HS256",    // 签名算法
  "typ": "JWT"       // 令牌类型
}
```

### 2️⃣ Payload（载荷）
```json
{
  "sub": "user123",           // Subject: 用户标识
  "exp": 1735689600,          // Expiration: 过期时间
  "iat": 1735603200,          // Issued At: 签发时间
  "name": "张三",              // 自定义声明
  "role": "admin"             // 自定义声明
}
```

### 3️⃣ Signature（签名）
```
HMACSHA256(
  base64UrlEncode(header) + "." + base64UrlEncode(payload),
  secret_key
)
```

> ⚠️ **注意**：Payload 只是 Base64 编码，**不是加密**！任何人都可以解码查看内容。

## 🛠️ 环境准备

In [ ]:
# 环境配置
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# 获取路径
notebook_path = Path.cwd()
backend_dir = notebook_path.parent.parent
root_dir = backend_dir.parent

# 添加 backend 到 sys.path
if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

# 加载测试环境变量
load_dotenv(root_dir / ".env.test", override=True)

print(f"✅ 环境: {os.getenv('ENVIRONMENT')}")
print(f"✅ Backend 路径: {backend_dir}")

In [ ]:
# 导入必要的库
import jwt  # PyJWT 库
import json
import base64
from datetime import datetime, timedelta, timezone

print("✅ PyJWT 版本:", jwt.__version__)

## 🔬 手动解析 JWT 结构

让我们先创建一个 JWT，然后手动解析它的三个部分：

In [ ]:
# 创建一个示例 JWT
SECRET_KEY = "my-super-secret-key-for-demo"
ALGORITHM = "HS256"

# 构建载荷
payload = {
    "sub": "user_12345",           # 用户 ID
    "name": "张三",                 # 用户名
    "role": "admin",               # 角色
    "iat": datetime.now(timezone.utc),  # 签发时间
    "exp": datetime.now(timezone.utc) + timedelta(hours=1)  # 1小时后过期
}

# 创建 JWT
token = jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)

print("🎫 生成的 JWT Token:")
print("=" * 80)
print(token)
print("=" * 80)
print(f"\n📏 Token 长度: {len(token)} 字符")

In [ ]:
# 手动解析 JWT 的三个部分
def decode_base64_url(data: str) -> str:
    """解码 Base64URL 编码的字符串"""
    # 添加必要的填充
    padding = 4 - len(data) % 4
    if padding != 4:
        data += "=" * padding
    # Base64URL 使用 - 和 _ 替代 + 和 /
    data = data.replace("-", "+").replace("_", "/")
    return base64.b64decode(data).decode("utf-8")

# 分割 token
parts = token.split(".")
header_b64, payload_b64, signature_b64 = parts

print("🔍 JWT 结构解析")
print("=" * 80)

# 解析 Header
print("\n1️⃣ HEADER (头部):")
print(f"   Base64: {header_b64}")
header_json = decode_base64_url(header_b64)
header = json.loads(header_json)
print(f"   解码后: {json.dumps(header, indent=4, ensure_ascii=False)}")

# 解析 Payload
print("\n2️⃣ PAYLOAD (载荷):")
print(f"   Base64: {payload_b64}")
payload_json = decode_base64_url(payload_b64)
payload_decoded = json.loads(payload_json)
print(f"   解码后: {json.dumps(payload_decoded, indent=4, ensure_ascii=False)}")

# Signature 无法解码为 JSON（是二进制数据）
print("\n3️⃣ SIGNATURE (签名):")
print(f"   Base64: {signature_b64}")
print(f"   （签名是 HMAC-SHA256 哈希值，无法解码为可读文本）")

print("\n" + "=" * 80)
print("⚠️ 注意：任何人都可以解码 Header 和 Payload！")
print("   JWT 不提供加密，只提供签名验证！")

## 🔑 PyJWT 库使用

### 创建和验证 Token

In [ ]:
# 创建带有过期时间的 Token
SECRET_KEY = "your-super-secret-key-keep-it-safe"  # 在生产环境中应该从环境变量读取
ALGORITHM = "HS256"

# 创建一个 1 小时后过期的 Token
payload = {
    "sub": "user123",              # Subject: 用户标识
    "email": "user@example.com",   # 自定义 claim
    "role": "admin",               # 自定义 claim: 用户角色
    "iat": datetime.utcnow(),      # Issued At: 签发时间
    "exp": datetime.utcnow() + timedelta(hours=1)  # Expiration: 过期时间
}

token = jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)
print("📝 生成的 Token:")
print(token)
print(f"\n🔢 Token 长度: {len(token)} 字符")

In [ ]:
# 验证并解码 Token
try:
    decoded = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
    print("✅ Token 验证成功!")
    print("\n📋 解码后的 Payload:")
    for key, value in decoded.items():
        print(f"  {key}: {value}")
except jwt.ExpiredSignatureError:
    print("❌ Token 已过期")
except jwt.InvalidTokenError as e:
    print(f"❌ 无效的 Token: {e}")

## ⏰ 过期处理

JWT 的一个重要特性是可以设置过期时间，让我们看看过期 Token 的行为：

In [ ]:
# 创建一个已经过期的 Token
expired_payload = {
    "sub": "user123",
    "exp": datetime.utcnow() - timedelta(hours=1)  # 1 小时前就过期了
}

expired_token = jwt.encode(expired_payload, SECRET_KEY, algorithm=ALGORITHM)
print("🕐 创建了一个已过期的 Token")
print(f"Token: {expired_token[:50]}...")

# 尝试验证过期的 Token
print("\n尝试验证过期的 Token:")
try:
    decoded = jwt.decode(expired_token, SECRET_KEY, algorithms=[ALGORITHM])
    print("✅ Token 有效")
except jwt.ExpiredSignatureError:
    print("❌ jwt.ExpiredSignatureError: Token 已过期!")
except jwt.InvalidTokenError as e:
    print(f"❌ 其他错误: {e}")

## 🛡️ 常见错误类型

PyJWT 提供了多种异常类型来处理不同的错误情况：

In [ ]:
def test_token_validation(token_str: str, secret: str, description: str):
    """测试 Token 验证并捕获各种异常"""
    print(f"\n🧪 测试: {description}")
    print(f"   Token: {token_str[:40]}..." if len(token_str) > 40 else f"   Token: {token_str}")
    try:
        decoded = jwt.decode(token_str, secret, algorithms=[ALGORITHM])
        print(f"   ✅ 成功: {decoded}")
    except jwt.ExpiredSignatureError:
        print("   ❌ ExpiredSignatureError: Token 已过期")
    except jwt.InvalidSignatureError:
        print("   ❌ InvalidSignatureError: 签名无效 (密钥不匹配)")
    except jwt.DecodeError:
        print("   ❌ DecodeError: Token 格式无效")
    except jwt.InvalidTokenError as e:
        print(f"   ❌ InvalidTokenError: {e}")

# 创建一个有效的 Token
valid_token = jwt.encode({"sub": "user123", "exp": datetime.utcnow() + timedelta(hours=1)}, SECRET_KEY, algorithm=ALGORITHM)

# 测试各种情况
test_token_validation(valid_token, SECRET_KEY, "有效的 Token")
test_token_validation(valid_token, "wrong-secret-key", "使用错误的密钥")
test_token_validation("invalid.token.here", SECRET_KEY, "格式无效的 Token")
test_token_validation(expired_token, SECRET_KEY, "过期的 Token")

## 🔄 JWT vs Session 对比

| 特性 | JWT | Session |
|------|-----|---------|
| **存储位置** | 客户端 (Cookie/LocalStorage) | 服务器端 (内存/Redis/数据库) |
| **扩展性** | ✅ 无状态，易于水平扩展 | ⚠️ 需要共享 Session 存储 |
| **性能** | ✅ 无需查询数据库 | ⚠️ 每次请求需要查询 Session |
| **安全性** | ⚠️ Token 无法撤销 | ✅ 可以随时销毁 Session |
| **大小** | ⚠️ 相对较大 | ✅ 仅存储 Session ID |
| **跨域** | ✅ 天然支持 | ⚠️ 需要特殊配置 |

## ⚠️ 安全最佳实践

### 1. 密钥管理
```python
# ❌ 不要这样做
SECRET_KEY = "my-secret-key"  # 硬编码在代码中

# ✅ 应该这样做
import os
SECRET_KEY = os.getenv("SECRET_KEY")  # 从环境变量读取
```

### 2. 选择合适的过期时间
- **Access Token**: 短期有效 (15分钟 - 1小时)
- **Refresh Token**: 长期有效 (7天 - 30天)

### 3. 使用 HTTPS
JWT 在传输过程中应该始终使用 HTTPS 加密

### 4. 不要在 Payload 中存储敏感信息
```python
# ❌ 不要存储密码、信用卡等敏感信息
payload = {"password": "123456"}  # 危险!

# ✅ 只存储必要的标识信息
payload = {"sub": "user_id", "role": "admin"}
```

### 5. 验证所有 Claims
```python
# 使用 options 参数强制验证
jwt.decode(token, SECRET_KEY, algorithms=["HS256"], 
           options={"require": ["exp", "sub"]})
```

## 📚 总结

1. **JWT 是无状态的认证方式**：服务器不需要存储 Session 信息
2. **三部分组成**：Header.Payload.Signature
3. **签名确保完整性**：修改任何内容都会导致签名验证失败
4. **过期机制**：使用 `exp` claim 控制 Token 有效期
5. **安全第一**：使用强密钥、HTTPS、短过期时间

接下来，在 `security.ipynb` 中我们将测试项目中实际的 security.py 模块功能！